<a href="https://colab.research.google.com/github/anhduong77/Tracking-System-Management/blob/main/notebook/1_Engineering_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
customers = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_customers_dataset.csv')
order_items = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_order_items_dataset.csv')
payments = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_order_payments_dataset.csv')
orders = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_orders_dataset.csv')
sellers = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_sellers_dataset.csv')
products = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_products_dataset.csv')
category_name = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/product_category_name_translation.csv')
geolocation = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_geolocation_dataset.csv')
reviews = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/olist_order_reviews_dataset.csv')

- Since database exist multiple data table, we need to select and merge the important features beforehand.

In [ ]:
conn = sqlite3.connect(":memory:")
customers.to_sql("customers", conn, index=False, if_exists="replace")
payments.to_sql("payments", conn, index=False, if_exists="replace")
reviews.to_sql("reviews", conn, index=False, if_exists="replace")
orders.to_sql("orders", conn, index=False, if_exists="replace")
sellers.to_sql("sellers", conn, index=False, if_exists="replace")
products.to_sql("products", conn, index=False, if_exists="replace")
category_name.to_sql("category_name", conn, index=False, if_exists="replace")
geolocation.to_sql("geolocation", conn, index=False, if_exists="replace")
order_items.to_sql("order_items", conn, index=False, if_exists="replace")

112650

In [ ]:
display(orders.head(2))
display(order_items.head(2))
display(sellers.head(2))


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


In [ ]:
query = """
WITH
-- 1. CLEAN UP GEOLOCATION (Remove duplicates by averaging lat/lng per Zip)
geo_agg AS (
    SELECT
        geolocation_zip_code_prefix,
        AVG(geolocation_lat) as lat,
        AVG(geolocation_lng) as lng
    FROM geolocation
    GROUP BY geolocation_zip_code_prefix
),

order_item_agg AS (
    SELECT
        order_id,
        COUNT(*) AS total_items,
        SUM(price) AS total_price,
        SUM(freight_value) AS total_freight,
        AVG(price) AS avg_item_price
    FROM order_items
    GROUP BY order_id
),

payment_agg AS (
    SELECT
        order_id,
        COUNT(*) AS payment_count,
        SUM(payment_value) AS total_payment_value,
        MAX(payment_installments) AS max_installments
    FROM payments
    GROUP BY order_id
),

review_agg AS (
    SELECT
        order_id,
        MAX(review_score) AS review_score,
        MAX(review_comment_title) AS review_comment_title,
        MAX(review_comment_message) AS review_comment_message,
        MAX(review_creation_date) AS review_creation_date
    FROM reviews
    GROUP BY order_id
)

SELECT
    -- Orders
    o.order_id,
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,

    -- Customer Location
    c.customer_zip_code_prefix,
    c.customer_city,
    c.customer_state,
    geo_c.lat AS customer_lat,  -- ADDED
    geo_c.lng AS customer_lng,  -- ADDED

    -- Seller Location (We take the seller from the raw items join)
    s.seller_city,
    s.seller_state,
    geo_s.lat AS seller_lat,    -- ADDED
    geo_s.lng AS seller_lng,    -- ADDED

    -- Aggregated Order Info
    oi.total_items,
    oi.total_price,
    oi.total_freight,

    -- Review & Payment
    r.review_score,
    pay.total_payment_value

FROM orders o
LEFT JOIN customers c
    ON o.customer_id = c.customer_id

-- JOIN FOR CUSTOMER COORDINATES
LEFT JOIN geo_agg geo_c
    ON c.customer_zip_code_prefix = geo_c.geolocation_zip_code_prefix

-- JOIN RAW ITEMS TO GET SELLER ID
LEFT JOIN order_items oi_raw
    ON o.order_id = oi_raw.order_id
    AND oi_raw.order_item_id = 1 -- Optimization: Only grab the 1st item to avoid duplicates

LEFT JOIN sellers s
    ON oi_raw.seller_id = s.seller_id

-- JOIN FOR SELLER COORDINATES
LEFT JOIN geo_agg geo_s
    ON s.seller_zip_code_prefix = geo_s.geolocation_zip_code_prefix

-- OTHER JOINS
LEFT JOIN order_item_agg oi
    ON o.order_id = oi.order_id
INNER JOIN review_agg r
    ON o.order_id = r.order_id
LEFT JOIN payment_agg pay
    ON o.order_id = pay.order_id
;

"""

In [ ]:
result_df = pd.read_sql(query, conn)

In [ ]:
result_df.to_csv("/content/drive/MyDrive/CustomerSatisfaction/archive/merged_data.csv", index=False)